In [7]:
import numpy
import os
import csv
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [8]:
#Define Paths
#pathto = 'Results/'
headline = (pd.read_excel('headerlist.xlsx', header=None, dtype=str)) #File containing names f vessels
csv_suffix_before = "_before.csv"
csv_suffix_after = "_after.csv"

flippers = [1,2,3,13] # segmentation from cranial to caudal
#to make first point be the most proximal

defEND = [1,3,5,6,8,9,10,11,12,13,14,16,18,19,20,21]
#Vessels with a defined end point

In [9]:
#functions

def findlength(linearray, end):
    '''
    calculates centerline length from points in csv file
    calculated using euclidean distance
    '''
    index=0
    lencarrier=0.0
    for i in linearray:
        a = linearray[index,:]
        b = linearray[index+1 ,:]
        index+=1
        dist = numpy.linalg.norm(b-a)
        lencarrier = lencarrier + dist
        if index == end-1:
            break
    return lencarrier

def pointfinder(length, edist, array, ende):
    '''
    Finds six equidistant points along the centerline.
    iterates along the centerline, calculating the
    distance between each point and the next.
    If it exceeds a multiple of the equidistance, 
    the position is added to the Poslist as 
    position of an equidistant point.
    '''
    Poslist = [0] 
    Pointlist = [array[0]]
    index = 0 
    lengthIterate = 0.0 
    DistIterate=edist

    for i in array:
        a = array[index, :] #these points are used to iterate
        b = array[index + 1, :] #along the centerline
        index = index + 1
        dist = numpy.linalg.norm(b - a)
        lengthIterate = lengthIterate + dist
        if lengthIterate >= DistIterate:
            #point is selected as soon as the length 
            Poslist.append(numpy.where(numpy.all(array == b, axis=1))[0][0])
            Pointlist.append(b)
            DistIterate = edist + DistIterate
        if index == ende - 1:
            break
        elif lengthIterate > length:
            break
    if len(Poslist)<6:
        Poslist.append(ende - 1)
        Pointlist.append(array[ende-1])
    return Poslist, Pointlist

def createvecs(ListA, ListB):
    ''' Creates vectors between the corresponding points of the six
    equidistant points by subtracting the point without capnoperitoneum
    from its capno counterpart. The vectors are returned both in their 
    original form and as their normed versions in order to prepare them f
    or circular statistics calculations'''
    iterate=0
    veclist=[]
    while iterate < 6: 
        veclist.append(ListA[iterate]-ListB[iterate])
        iterate += 1
    norms = numpy.linalg.norm(veclist, axis=1, keepdims=True)
    unit_vecs = veclist / norms
    return unit_vecs, veclist

def circsingle(vectors):
    '''Calculates the Azimuth and Polar Angles from a list of vectors, 
    creates lists of the calculated angles and returns said lists'''
    azimuth = numpy.arctan2(vectors[:, 1], vectors[:, 0])  # Azimuth in xy plane
    polar = numpy.arccos(vectors[:, 2])  # Polar Angle in zx plane

    return azimuth, polar

def circvarstd(angles):
    '''Calculates the mean angle with circular variance and standard 
    deviation from a list of vectors and returns these values'''
    cosAn = numpy.cos(angles)
    sinAn = numpy.sin(angles)

    mean_cos = numpy.mean(cosAn)
    mean_sin = numpy.mean(sinAn)

    var=circular_variance = 1 - numpy.sqrt(mean_cos**2 + mean_sin**2)

    std=circular_std = numpy.sqrt(-2 * numpy.log(1 - circular_variance))
    
    return var, std

In [14]:
vesselit = 1
while vesselit < 25: #capped at number of vessels
    #define paths for vessel 
    if vesselit<12: #arteries
        csv_after = ''.join(['Centerlines/ArtC/',str(vesselit),'_after.csv'])
        csv_before = ''.join(['Centerlines/ArtO/',str(vesselit),'_before.csv'])
    else:#veins, ureters and esophagus
        csv_after = ''.join(['Centerlines/VeinsC/',str(vesselit),'_after.csv'])
        csv_before = ''.join(['Centerlines/VeinsO/',str(vesselit),'_before.csv'])

    #read data from csv
    before = numpy.genfromtxt(csv_before, delimiter = ";", encoding='UTF-8', skip_header=1)
    after = numpy.genfromtxt(csv_after, delimiter =";", encoding='UTF-8', skip_header=1)
    
    #flip centerlines that were segmented from distal to proximal
    if vesselit in flippers:
        before = numpy.flipud(before)
        after = numpy.flipud(after)

    #save length of pointlist to variable
    endB = before.shape[0]
    endA = after.shape[0]

    #use of findlength function defined above to find centerline length in mm
    lengthB=findlength(before,endB)
    lengthA=findlength(after,endA)
    
    #was no defined endpoint found during segmentation? (e.g. A.lienalis)   
    if (vesselit in defEND)==False:

        #comparing lengths of the vessels, in order to compare the same parts of the vessel as there is no significant points for comparison
        if lengthB >= lengthA:
            length = lengthA
        elif lengthB == lengthA:
            length = lengthA
        else:
            length = lengthB

        #Calculation of length of 6 equal parts of the length
        Edist = length / 5
        Edistc = Edist

        #pointfinder to identify equidistant points
        posA, pointsA = pointfinder(length=length, edist=Edist, array=after, ende=endA)
        posB, pointsB = pointfinder(length=length, edist=Edist, array=before, ende=endB)
        LengthA = length
        LengthB = length

    #pointfinding with defined length
    else:
        EdistA=lengthA/5
        EdistB=lengthB/5

        posA, pointsA = pointfinder(length=lengthA, edist=EdistA, array=after, ende=endA)
        posB, pointsB = pointfinder(length=lengthB, edist=EdistB, array=before, ende=endB)

    norm_vectors, org_vectors = createvecs(pointsA,pointsB)
    distlist = numpy.linalg.norm(org_vectors, axis=1)

    azlist, polist = circsingle(vectors)
    
    title = str(headline[0][vesselit-1])
    print(vesselit, title)

    resframe = pd.DataFrame({
    'Points After': [list(p) for p in numpy.round(pointsA,2)],
    'Positions A': posA,
    'Points Before': [list(p) for p in numpy.round(pointsB,2)],
    'Positions B': posB,
    'Distances in mm' : distlist,
    'Vectors Normed': [list(v) for v in numpy.round(norm_vectors,2)],
    'Vectors Original': [list(v) for v in numpy.round(org_vectors,2)],
    'Azimuth': numpy.round(numpy.degrees(azlist),2),
    'Polar': numpy.round(numpy.degrees(polist),2)
    }, index=range(1, len(pointsA)+1))  

    display(HTML(str(vesselit)+' '+str(headline[0][vesselit-1])))
    display(resframe)
    
    resframe.to_excel(''.join('Results/'+title+'.xlsx'))

    vesselit+=1

# positionen, längen, equidistanzen und punkte in einen dataframe; = sachen anpassen mittels leerzeichen

1 Truncus coeliacus


Points After  Positions A               Points Before  \
1  [20.06, -176.49, -1351.18]            0  [18.95, -181.48, -1349.71]   
2  [22.75, -177.02, -1355.53]            5  [20.81, -182.56, -1354.27]   
3  [24.56, -177.57, -1357.94]            8   [22.83, -183.4, -1357.68]   
4  [26.31, -180.06, -1360.61]           12  [25.11, -185.66, -1361.46]   
5  [26.67, -183.99, -1361.65]           16   [25.73, -189.17, -1363.3]   
6  [25.74, -187.96, -1361.47]           20   [25.56, -193.12, -1363.6]   

   Positions B  Distances in mm       Vectors Normed     Vectors Original  \
1            0         5.326336  [0.21, 0.94, -0.28]   [1.12, 5.0, -1.47]   
2            5         5.996811  [0.32, 0.92, -0.21]  [1.94, 5.53, -1.25]   
3            9         6.087041  [0.28, 0.96, -0.04]  [1.73, 5.83, -0.26]   
4           14         5.793969   [0.21, 0.97, 0.15]     [1.2, 5.6, 0.85]   
5           18         5.522645    [0.17, 0.94, 0.3]   [0.94, 5.19, 1.65]   
6           22         5.586247   [0.03, 0.92, 0.38]   [0.18, 5.16, 2.14]   

   Azimuth  Polar  
1   142.48  95.55  
2   109.46  65.15  
3   145.29  49.47  
4   143.29  50.40  
5    76.91  42.28  
6   148.02  70.73

2 A. gastrica sinistra


Points After  Positions A               Points Before  \
1  [25.86, -187.94, -1356.96]            0  [27.19, -193.89, -1360.77]   
2  [22.31, -189.61, -1353.94]            5  [23.01, -195.77, -1356.99]   
3  [18.47, -189.29, -1350.74]           10  [19.92, -196.03, -1354.45]   
4  [15.62, -189.16, -1346.69]           15  [16.38, -195.98, -1350.89]   
5  [14.77, -189.82, -1340.91]           21  [13.93, -196.06, -1346.56]   
6  [15.89, -190.73, -1336.22]           26  [13.33, -197.28, -1341.82]   

   Positions B  Distances in mm       Vectors Normed     Vectors Original  \
1            0         7.188446  [-0.19, 0.83, 0.53]  [-1.33, 5.95, 3.81]   
2            6         6.909162   [-0.1, 0.89, 0.44]  [-0.69, 6.16, 3.05]   
3           10         7.833177  [-0.19, 0.86, 0.47]  [-1.45, 6.74, 3.71]   
4           15         8.045343  [-0.09, 0.85, 0.52]   [-0.76, 6.82, 4.2]   
5           20         8.465085    [0.1, 0.74, 0.67]   [0.84, 6.24, 5.65]   
6           25         8.984870   [0.28, 0.73, 0.62]   [2.56, 6.55, 5.59]   

   Azimuth  Polar  
1   142.48  95.55  
2   109.46  65.15  
3   145.29  49.47  
4   143.29  50.40  
5    76.91  42.28  
6   148.02  70.73

3 A. hepatica communis


...

Points After  Positions A               Points Before  \
1  [-4.27, -180.24, -1200.62]            0    [-4.46, -185.6, -1198.0]   
2  [-3.46, -178.83, -1208.35]            8  [-2.91, -182.41, -1204.66]   
3   [-3.35, -177.3, -1217.12]           17  [-2.86, -179.92, -1212.22]   
4   [-3.11, -176.35, -1224.0]           24   [-2.16, -178.8, -1220.11]   
5   [-2.62, -175.0, -1231.66]           32  [-1.28, -177.62, -1227.94]   
6   [-1.62, -171.7, -1238.72]           40  [-0.86, -176.18, -1235.78]   

   Positions B  Distances in mm        Vectors Normed      Vectors Original  \
1            0         5.969795    [0.03, 0.9, -0.44]   [0.19, 5.36, -2.62]   
2            8         5.176531  [-0.11, 0.69, -0.71]  [-0.55, 3.58, -3.69]   
3           16         5.576681  [-0.09, 0.47, -0.88]   [-0.49, 2.62, -4.9]   
4           24         4.693827   [-0.2, 0.52, -0.83]  [-0.94, 2.45, -3.89]   
5           32         4.741003  [-0.28, 0.55, -0.78]  [-1.34, 2.62, -3.72]   
6           40         5.413354  [-0.14, 0.83, -0.54]  [-0.77, 4.48, -2.94]   

   Azimuth  Polar  
1   142.48  95.55  
2   109.46  65.15  
3   145.29  49.47  
4   143.29  50.40  
5    76.91  42.28  
6   148.02  70.73

23 Ureter left


Points After  Positions A               Points Before  \
1   [66.38, -188.71, -1467.4]            0  [62.59, -193.65, -1459.94]   
2  [52.39, -199.96, -1495.14]           39  [46.93, -201.05, -1485.02]   
3   [48.0, -203.92, -1523.99]           77  [54.69, -206.96, -1518.04]   
4  [33.66, -199.01, -1553.93]          115  [39.43, -205.92, -1550.08]   
5  [28.11, -197.11, -1590.72]          154  [36.13, -205.71, -1586.42]   
6  [35.86, -187.01, -1624.32]          192  [35.53, -201.39, -1622.69]   

   Positions B  Distances in mm        Vectors Normed      Vectors Original  \
1            0         9.717345   [0.39, 0.51, -0.77]   [3.79, 4.94, -7.46]   
2           39        11.549013   [0.47, 0.09, -0.88]  [5.46, 1.09, -10.12]   
3           77         9.458793  [-0.71, 0.32, -0.63]  [-6.69, 3.04, -5.96]   
4          116         9.796205  [-0.59, 0.71, -0.39]  [-5.77, 6.92, -3.85]   
5          154        12.521764  [-0.64, 0.69, -0.34]    [-8.02, 8.6, -4.3]   
6          192        14.476323   [0.02, 0.99, -0.11]  [0.32, 14.38, -1.63]   

   Azimuth  Polar  
1   142.48  95.55  
2   109.46  65.15  
3   145.29  49.47  
4   143.29  50.40  
5    76.91  42.28  
6   148.02  70.73

24 Ureter right


Points After  Positions A                Points Before  \
1   [-23.25, -163.5, -1453.71]            0  [-12.63, -171.66, -1452.41]   
2  [-18.18, -169.84, -1488.33]           45   [-14.5, -180.25, -1493.44]   
3  [-12.67, -181.15, -1522.41]           90   [-2.73, -188.04, -1532.75]   
4    [-6.34, -185.07, -1562.4]          135     [3.08, -192.1, -1572.13]   
5    [2.56, -180.61, -1602.75]          179    [0.98, -187.38, -1610.39]   
6   [-0.21, -183.11, -1644.54]          224   [14.44, -192.26, -1650.58]   

   Positions B  Distances in mm       Vectors Normed      Vectors Original  \
1            0        13.456746  [-0.79, 0.61, -0.1]  [-10.62, 8.16, -1.3]   
2           45        12.165052   [-0.3, 0.86, 0.42]  [-3.68, 10.41, 5.11]   
3           90        15.919382  [-0.62, 0.43, 0.65]  [-9.95, 6.89, 10.35]   
4          135        15.261608  [-0.62, 0.46, 0.64]   [-9.43, 7.03, 9.73]   
5          180        10.331043   [0.15, 0.66, 0.74]    [1.57, 6.77, 7.64]   
6          224        18.297044    [-0.8, 0.5, 0.33]  [-14.65, 9.15, 6.04]   

   Azimuth  Polar  
1   142.48  95.55  
2   109.46  65.15  
3   145.29  49.47  
4   143.29  50.40  
5    76.91  42.28  
6   148.02  70.73

In [7]:
#hier dann circstats berechnen - einfach winkel über ein gefäß und da kleine diagramme machen und ausgeben mit matplotlib.
#zeug wieder aus dem dataframe nehmen oder mittels positionen holen. oder gleich oben in der schleife machen?
